In [1]:
import torch
import mmcv
from mmpose.apis import MMPoseInferencer

print(f"PyTorch: {torch.__version__} (CUDA: {torch.version.cuda})")
print(f"MMCV: {mmcv.__version__}")
try:
    from mmcv.ops import active_rotated_filter
    print("Test load _ext: THÀNH CÔNG! (Đã nạp được C++ Extension)")
except Exception as e:
    print(f"Test load _ext: THẤT BẠI. Lỗi: {e}")

PyTorch: 2.1.0+cu118 (CUDA: 11.8)
MMCV: 2.1.0
Test load _ext: THÀNH CÔNG! (Đã nạp được C++ Extension)


In [3]:
import torch
import sys

print("--- KẾT QUẢ KIỂM TRA MÔI TRƯỜNG ---")
print(f"Python: {sys.version.split()[0]}")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.version.cuda}")
print(f"Có GPU không?: {torch.cuda.is_available()}")

# Logic tạo lệnh cài đặt
try:
    torch_ver = torch.__version__.split('+')[0] # ví dụ 2.1.0
    cuda_ver = torch.version.cuda.replace('.', '') # ví dụ 118
    
    # URL kho chứa wheel của OpenMMLab
    base_url = "https://download.openmmlab.com/mmcv/dist"
    
    # Tạo link tìm kiếm (find-links)
    find_link = f"{base_url}/cu{cuda_ver}/torch{torch_ver}/index.html"
    
    print("\n>>> HÃY CHẠY 2 LỆNH SAU TRONG TERMINAL (ANACONDA PROMPT):")
    print("-" * 60)
    print("pip uninstall mmcv -y")
    print(f'pip install mmcv==2.1.0 -f {find_link}')
    print("-" * 60)
    
except Exception as e:
    print(f"\nLỗi khi tạo link: {e}")
    print("Có thể PyTorch của bạn là bản CPU hoặc phiên bản quá mới/cũ chưa được hỗ trợ.")

--- KẾT QUẢ KIỂM TRA MÔI TRƯỜNG ---
Python: 3.9.25
PyTorch: 2.7.1+cu118
CUDA: 11.8
Có GPU không?: True

>>> HÃY CHẠY 2 LỆNH SAU TRONG TERMINAL (ANACONDA PROMPT):
------------------------------------------------------------
pip uninstall mmcv -y
pip install mmcv==2.1.0 -f https://download.openmmlab.com/mmcv/dist/cu118/torch2.7.1/index.html
------------------------------------------------------------


In [2]:
import os
import zipfile
import shutil
import glob

# 1. Cấu hình thư mục
data_root = 'data/lsp'
os.makedirs(data_root, exist_ok=True)

# 2. Tìm file zip đã tải thủ công
# Tìm bất kỳ file .zip nào ở thư mục hiện tại
zip_files = glob.glob('*.zip') 

if not zip_files:
    print("LỖI: Không tìm thấy file .zip nào!")
    print("Bạn vui lòng tải dataset từ Kaggle và upload file .zip vào thư mục này.")
else:
    # Lấy file zip đầu tiên tìm thấy
    zip_path = zip_files[0]
    print(f"Đã tìm thấy file: {zip_path}. Đang tiến hành giải nén...")

    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(data_root)
        print("Giải nén thành công!")
        
        # 3. Xử lý cấu trúc thư mục (Vì Kaggle thường lồng thư mục)
        # Kiểm tra xem ảnh nằm ở đâu
        images_path = os.path.join(data_root, 'images')
        
        # Trường hợp 1: Giải nén xong thấy thư mục 'lsp_dataset' bên trong (cấu trúc gốc)
        sub_dir = os.path.join(data_root, 'lsp_dataset')
        if os.path.exists(sub_dir):
            # Di chuyển nội dung ra ngoài data_root
            for item in os.listdir(sub_dir):
                shutil.move(os.path.join(sub_dir, item), data_root)
            os.rmdir(sub_dir)
            print("Đã sắp xếp lại cấu trúc thư mục.")

        # Trường hợp 2: Kaggle đặt tên folder là 'leeds_sports_pose'
        sub_dir_kaggle = os.path.join(data_root, 'leeds_sports_pose', 'leeds_sports_pose')
        if os.path.exists(sub_dir_kaggle):
             for item in os.listdir(sub_dir_kaggle):
                shutil.move(os.path.join(sub_dir_kaggle, item), data_root)
             print("Đã xử lý cấu trúc thư mục Kaggle.")

        # Kiểm tra cuối cùng
        if os.path.exists(os.path.join(data_root, 'joints.mat')):
             print("\nHOÀN TẤT! Dữ liệu đã sẵn sàng tại: data/lsp")
             print("Bạn có thể chạy tiếp phần code Đánh giá (Evaluation).")
        else:
             print("\nCẢNH BÁO: Không tìm thấy file joints.mat. Hãy kiểm tra lại thư mục data/lsp")

    except zipfile.BadZipFile:
        print("Lỗi: File zip bị hỏng. Vui lòng tải lại.")

Đã tìm thấy file: archive (4).zip. Đang tiến hành giải nén...
Giải nén thành công!

HOÀN TẤT! Dữ liệu đã sẵn sàng tại: data/lsp
Bạn có thể chạy tiếp phần code Đánh giá (Evaluation).


In [3]:
import numpy as np
import scipy.io as sio
import os
import cv2
from mmpose.apis import MMPoseInferencer
from tqdm import tqdm

# 1. CẤU HÌNH
DATA_DIR = 'data/lsp'
IMG_DIR = os.path.join(DATA_DIR, 'images')
GT_PATH = os.path.join(DATA_DIR, 'joints.mat')

# Chọn model RTMPose-m (Medium) train trên COCO
# MMPose sẽ tự động tải checkpoint về
model_name = 'rtmpose-m_8xb256-420e_coco-256x192'

# 2. HÀM MAPPING KEYPOINTS (COCO 17 điểm -> LSP 14 điểm)
# Chúng ta chỉ so sánh 12 khớp chung (bỏ qua mắt, tai của COCO và Head-top/Neck của LSP để đơn giản hoá)
# Thứ tự LSP: R Ankle(0), R Knee(1), R Hip(2), L Hip(3), L Knee(4), L Ankle(5), R Wrist(6), R Elbow(7), R Shoulder(8), L Shoulder(9), L Elbow(10), L Wrist(11), Neck(12), Head(13)
# Thứ tự COCO: Nose(0), Eye(1-2), Ear(3-4), Shoulder(5-6), Elbow(7-8), Wrist(9-10), Hip(11-12), Knee(13-14), Ankle(15-16)

# Map: LSP index -> COCO index
joint_map = {
    0: 16, # R Ankle
    1: 14, # R Knee
    2: 12, # R Hip
    3: 11, # L Hip
    4: 13, # L Knee
    5: 15, # L Ankle
    6: 10, # R Wrist
    7: 8,  # R Elbow
    8: 6,  # R Shoulder
    9: 5,  # L Shoulder
    10: 7, # L Elbow
    11: 9  # L Wrist
}
eval_indices = list(joint_map.keys()) # Chỉ đánh giá các khớp có trong map

# 3. HÀM TÍNH PCK (Percentage of Correct Keypoints)
def calculate_pck(predictions, ground_truths, threshold=0.2):
    """
    Tính PCK dựa trên đường chéo bounding box của thân người (Torso Diameter)
    """
    correct_count = 0
    total_count = 0

    for pred, gt in zip(predictions, ground_truths):
        # Lấy các khớp vai và hông để tính kích thước thân người
        # LSP: R Hip(2), L Hip(3), R Shoulder(8), L Shoulder(9)
        if gt.shape[1] > 2 and gt[2, 2] == 0: continue # Skip nếu không hiện hữu (visible)

        # Tính kích thước tham chiếu (Torso diameter)
        r_shoulder = gt[8, :2]
        l_hip = gt[3, :2]
        torso_size = np.linalg.norm(r_shoulder - l_hip)
        if torso_size == 0: torso_size = 100 # Fallback

        thr = threshold * torso_size

        for lsp_idx in eval_indices:
            # Kiểm tra xem khớp có visible trong GT không (cột thứ 3 của LSP là invisible flag?)
            # Lưu ý: LSP joints shape là (3, 14, 2000). Cột 3: 0: hidden, 1: visible
            # Ở đây ta lấy 100 ảnh đầu để demo nhanh
            pass

            coco_idx = joint_map[lsp_idx]
            
            # Lấy toạ độ
            pred_pt = pred[coco_idx][:2] # x, y
            gt_pt = gt[lsp_idx][:2]      # x, y

            # Tính khoảng cách
            dist = np.linalg.norm(pred_pt - gt_pt)
            
            if dist <= thr:
                correct_count += 1
            total_count += 1
            
    return correct_count / total_count if total_count > 0 else 0

# 4. CHẠY THỰC NGHIỆM
print("Khởi tạo model RTMPose...")
inferencer = MMPoseInferencer(model_name)

print("Đang đọc Ground Truth...")
# Load file mat: shape gốc là (3, 14, 2000) -> transpose thành (2000, 14, 3) cho dễ dùng
joints_gt = sio.loadmat(GT_PATH)['joints'].transpose(2, 1, 0) 

# Chỉ test trên 100 ảnh đầu tiên để tiết kiệm thời gian (có thể tăng lên 2000 nếu muốn full)
NUM_TEST = 2000 
preds_list = []
gt_list = []

print(f"Bắt đầu chạy Inference trên {NUM_TEST} ảnh...")
for i in tqdm(range(NUM_TEST)):
    img_name = f"im{i+1:04d}.jpg" # Tên file ảnh LSP: im0001.jpg, im0002.jpg...
    img_path = os.path.join(IMG_DIR, img_name)
    
    if not os.path.exists(img_path): continue

    # Chạy Inference
    result_generator = inferencer(img_path, return_vis=False)
    result = next(result_generator)
    
    # Lấy danh sách người detect được trong ảnh đầu tiên
    detected_people = result['predictions'][0] # Đây là một LIST
    
    # Kiểm tra xem có detect được ai không
    if not detected_people:
        print(f"Bỏ qua {img_name}: Không tìm thấy người.")
        continue
        
    # Lấy người đầu tiên (thường là người chính giữa hoặc rõ nhất trong LSP)
    first_person = detected_people[0] # Đây mới là DICT
    
    # Lấy keypoints của người đó
    keypoints = np.array(first_person['keypoints'])
    preds_list.append(keypoints)
    
    # Lấy Ground Truth tương ứng (nhớ đổi shape nếu cần thiết ở bước load mat file)
    gt_list.append(joints_gt[i])

# 5. KẾT QUẢ
pck_score = calculate_pck(preds_list, gt_list, threshold=0.2)
print("\n" + "="*30)
print(f"KẾT QUẢ ĐÁNH GIÁ TRÊN TẬP LSP (Subset {NUM_TEST} ảnh)")
print(f"Model: {model_name}")
print(f"Metric: PCK@0.3 (trên các khớp tay chân chính)")
print(f"Accuracy: {pck_score * 100:.2f}%")
print("="*30)

Khởi tạo model RTMPose...
Loads checkpoint by http backend from path: https://download.openmmlab.com/mmpose/v1/projects/rtmposev1/rtmpose-m_simcc-coco_pt-aic-coco_420e-256x192-d8dd5ca4_20230127.pth
Loads checkpoint by http backend from path: https://download.openmmlab.com/mmpose/v1/projects/rtmposev1/rtmdet_m_8xb32-100e_coco-obj365-person-235e8209.pth
Đang đọc Ground Truth...
Bắt đầu chạy Inference trên 2000 ảnh...


100%|██████████| 2000/2000 [02:30<00:00, 13.28it/s]


KẾT QUẢ ĐÁNH GIÁ TRÊN TẬP LSP (Subset 2000 ảnh)
Model: rtmpose-m_8xb256-420e_coco-256x192
Metric: PCK@0.3 (trên các khớp tay chân chính)
Accuracy: 78.53%
